In [1]:
import os
import pandas as pd
import numpy as np
import pickle
from datetime import datetime,timedelta
import re
import math
import matplotlib.pyplot as plt

In [2]:
#'''文件路径定义'''
ACTION_FILE = './datafile/JData_Action.csv'
COMMENT_FILE = './datafile/JData_Comment.csv'
PRODUCT_FILE = './datafile/JData_Product.csv'
USER_FILE = './datafile/JData_User.csv'

# 5.15号代码，转换思路

In [104]:
# with open('./cache/Action_2016-02-01_2016-04-11-user_id-cate-time-type','rb') as f:
#     train_dataset = pickle.load(f)
# buy_date = train_dataset[train_dataset['type']==4]
# buy_date = buy_date.sort_values(by='user_id',ascending=True)
# # 选出最后购买商品的时间作为起始日期
# buy_date = buy_date[['user_id','time']]
# buy_date['time'] = buy_date.time.apply(lambda t:t[:10])
# buy_date = buy_date.groupby('user_id',as_index=False).time.max()
# buy_date.columns = ['user_id','last_buy_time']
# train_dataset_1 = train_dataset[['user_id','time']]
# train_dataset_1['time'] = train_dataset_1.time.apply(lambda t:t[:10])
# train_dataset_1 = train_dataset_1.drop_duplicates()
# buy_date_1 = pd.merge(train_dataset_1,buy_date,on='user_id',how='left')
# buy_date_1 = buy_date_1.fillna('2016-01-30')
#buy_date_1['time'] = buy_date_1.time.apply(lambda t:t[:10])
# buy_date_1 = buy_date_1.drop_duplicates()
# buy_date_1 = buy_date_1[buy_date_1['time']>buy_date_1['last_buy_time']]
# type_dummies = pd.get_dummies(buy_date_1['time'],prefix=None)
# buy_date_2 = pd.concat([buy_date_1,type_dummies],axis=1)
# buy_date_2 = buy_date_2.drop(['time','last_buy_time'],axis=1)
# # buy_date_2.to_csv('new_load_frame.csv',index=False)
# buy_date_2 = buy_date_2.groupby('user_id',as_index=False).sum()
# buy_date_2['load_days'] = buy_date_2.sum(axis=1) - buy_date_2['user_id']
# buy_date_2 = pd.merge(buy_date_2,buy_date_1,on='user_id',how='left')
# buy_date_3 = buy_date_2.drop(['time'],axis=1)
# buy_date_3 = buy_date_3.drop_duplicates()
# buy_date_3[buy_date_3['last_buy_time']=='2016-01-30']             #总共10万多用户，就要接近8万个用户这两个月来没买过一件商品
# buy_date_3.to_csv('load_frame.csv',index=False)
# buy_date_3 = pd.read_csv('load_frame.csv')
# simple_load_user = buy_date_3[['user_id','2016-04-04','2016-04-05','2016-04-06','2016-04-07','2016-04-08','2016-04-09','2016-04-10','load_days','last_buy_time']]
# test_set = pd.read_csv('test_set.csv')
# test_set_user_load = pd.merge(test_set,simple_load_user,on='user_id',how='left')
# test_set_user_load[test_set_user_load.last_buy_time=='2016-01-30']            #在4.10前浏览的1172个，这两个月没买的就占了937个
# test_set_user_load_1 = test_set_user_load.dropna(axis=0)
# test_set_user_load_1 = test_set_user_load_1.sort_values(by='load_days',ascending=True)
# load_one_day_user = test_set_user_load_1[test_set_user_load_1['load_days']==1]              #两个月没买，登陆天数超过20天的用户购买数只有31个
# load_one_day_user.sort_values(by='last_buy_time',ascending=True)
# load_10 = load_one_day_user[load_one_day_user['2016-04-10']==1]                           #56个，总数是233个
# load_9 = load_one_day_user[load_one_day_user['2016-04-09']==1]                              #23个
# load_8 = load_one_day_user[load_one_day_user['2016-04-08']==1]                                #26个
# load_7 = load_one_day_user[load_one_day_user['2016-04-07']==1]                                  #11个
# load_6 = load_one_day_user[load_one_day_user['2016-04-06']==1]                                    #7个
# load_5 = load_one_day_user[load_one_day_user['2016-04-05']==1]                                #7个
# load_4 = load_one_day_user[load_one_day_user['2016-04-04']==1]                                  #6个
# len(load_4)
# #last_7_day_not_load = load_one_day_user[(load_one_day_user['2016-04-04']==0)&(load_one_day_user['2016-04-05']==0)&\
#  #                                       (load_one_day_user['2016-04-06']==0)&(load_one_day_user['2016-04-07']==0)&\
#  #                                       (load_one_day_user['2016-04-08']==0)&(load_one_day_user['2016-04-09']==0)&\
#  #                                       (load_one_day_user['2016-04-10']==0)][['user_id','last_buy_time']]
# #last_7_day_not_load[last_7_day_not_load['last_buy_time']=='2016-01-30']             #85个
# #last_7_day_not_load = pd.merge(last_7_day_not_load,train_dataset,on='user_id',how='left')
# #last_7_day_not_load
# load_less_8_days = test_set_user_load_1[test_set_user_load_1['load_days']<=7] 
# less_8_all_user = pd.DataFrame(simple_load_user[simple_load_user['load_days']<=7]['user_id'])
# train_dataset_2 = train_dataset[['user_id','cate','type']]
# less_8_all_user_dataset = pd.merge(less_8_all_user,train_dataset_2,how='left',on='user_id')
# less_8_all_user_dataset_groupby = less_8_all_user_dataset.groupby(['user_id','cate'],as_index=False).count()
# load_three_day_user = test_set_user_load_1[test_set_user_load_1['load_days']<=3] 
load_three_day_user[load_three_day_user['last_buy_time']=='2016-01-30']

,user_id,sku_id,2016-04-04,2016-04-05,2016-04-06,2016-04-07,2016-04-08,2016-04-09,2016-04-10,load_days,last_buy_time
208,277004,153395,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,2016-01-30
1245,271566,5505,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2016-01-30
1243,237139,145946,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,2016-01-30
606,208043,20308,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,2016-01-30
1229,280143,166354,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2016-01-30
223,255045,131300,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2016-01-30
596,297118,61531,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2016-01-30
594,295928,73469,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2016-01-30
588,228457,145990,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2016-01-30
231,299744,132782,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2016-01-30


In [1]:
# load_less_4_days = buy_date_3[buy_date_3['load_days']<=3]
# load_less_4_days = pd.merge(load_less_4_days,user_info,on='user_id',how='left')
# temp_1 = load_less_4_days[(load_less_4_days['user_lv_cd']>2)&((load_less_4_days['age']=='36-45岁')|\
#                                                      (load_less_4_days['age']=='26-35岁')|(load_less_4_days['age']=='-1'))]
# temp_2 = temp_1[temp_1['last_buy_time']=='2016-01-30']       #7599
# temp_2[(temp_2['2016-04-04']==1)|(temp_2['2016-04-05']==1)|(temp_2['2016-04-06']==1)|(temp_2['2016-04-07']==1)|(temp_2['2016-04-08']==1)|\
#                     (temp_2['2016-04-09']==1)|(temp_2['2016-04-10']==1)]   #2877
# temp_3 = pd.merge(temp_2,load_three_day_user,on='user_id',how='left').dropna(axis=0)       #447
# have_cate_8_browse_user = pd.read_csv('have_cate_8_browse_record.csv')
# temp_4 = pd.merge(temp_2,have_cate_8_browse_user,on='user_id',how='left').dropna(axis=0)


# 引入用户数据

In [107]:
# user_info = pd.read_csv(USER_FILE,encoding='gbk')
# load_less_8_days_user_info = pd.merge(test_set_user_load,user_info,how='left',on='user_id')  
#load_less_8_days_user_info[load_less_8_days_user_info['user_lv_cd']>2] #1385个用户中有1371个是用户等级大于2, 588个等级为5,4有491个，3有295个
# load_less_8_days_user_info[load_less_8_days_user_info['age']=='-1']    #201
# load_less_8_days_user_info[load_less_8_days_user_info['age']=='16-25岁'] #107
# load_less_8_days_user_info[load_less_8_days_user_info['age']=='26-35岁']  #630
# load_less_8_days_user_info[load_less_8_days_user_info['age']=='36-45岁']  #394
# load_less_8_days_user_info[load_less_8_days_user_info['age']=='46-55岁']  #28 
# load_less_8_days_user_info[load_less_8_days_user_info['user_lv_cd']==5]     #585
# load_less_8_days_user_info[load_less_8_days_user_info['user_lv_cd']==4]      #491
# load_less_8_days_user_info[load_less_8_days_user_info['user_lv_cd']==3]      #295
# load_less_8_days_user_info[(load_less_8_days_user_info['user_lv_cd']==5)&(load_less_8_days_user_info['age']=='-1')]       #87
# load_less_8_days_user_info[(load_less_8_days_user_info['user_lv_cd']==5)&(load_less_8_days_user_info['age']=='16-25岁')]  #31
# load_less_8_days_user_info[(load_less_8_days_user_info['user_lv_cd']==5)&(load_less_8_days_user_info['age']=='26-35岁')]  #293
# load_less_8_days_user_info[(load_less_8_days_user_info['user_lv_cd']==5)&(load_less_8_days_user_info['age']=='36-45岁')]  #140
# load_less_8_days_user_info[(load_less_8_days_user_info['user_lv_cd']==5)&(load_less_8_days_user_info['age']=='46-55岁')]  #16

# load_less_8_days_user_info[(load_less_8_days_user_info['user_lv_cd']==4)&(load_less_8_days_user_info['age']=='-1')]     #52
# load_less_8_days_user_info[(load_less_8_days_user_info['user_lv_cd']==4)&(load_less_8_days_user_info['age']=='16-25岁')]  #46
# load_less_8_days_user_info[(load_less_8_days_user_info['user_lv_cd']==4)&(load_less_8_days_user_info['age']=='26-35岁')]  #248
# load_less_8_days_user_info[(load_less_8_days_user_info['user_lv_cd']==4)&(load_less_8_days_user_info['age']=='36-45岁')]  #134
# load_less_8_days_user_info[(load_less_8_days_user_info['user_lv_cd']==4)&(load_less_8_days_user_info['age']=='46-55岁')] #8

# load_less_8_days_user_info[(load_less_8_days_user_info['user_lv_cd']==3)&(load_less_8_days_user_info['age']=='-1')]      #54
# load_less_8_days_user_info[(load_less_8_days_user_info['user_lv_cd']==3)&(load_less_8_days_user_info['age']=='16-25岁')]  #28
# load_less_8_days_user_info[(load_less_8_days_user_info['user_lv_cd']==3)&(load_less_8_days_user_info['age']=='26-35岁')] #88
# load_less_8_days_user_info[(load_less_8_days_user_info['user_lv_cd']==3)&(load_less_8_days_user_info['age']=='36-45岁')] #117
# load_less_8_days_user_info[(load_less_8_days_user_info['user_lv_cd']==3)&(load_less_8_days_user_info['age']=='26-35岁')] #4

# load_less_8_days_user_info[(load_less_8_days_user_info['user_lv_cd']>=3)&((load_less_8_days_user_info['age']=='36-45岁')|\
#                            (load_less_8_days_user_info['age']=='26-35岁')|\load_less_8_days_user_info['age']=='-1'))]  # 1213
load_less_8_days_user_info[(load_less_8_days_user_info['2016-04-10']==1)|(load_less_8_days_user_info['2016-04-09']==1)|\
                           (load_less_8_days_user_info['2016-04-08']==1)|(load_less_8_days_user_info['2016-04-07']==1)|\
                          (load_less_8_days_user_info['2016-04-06']==1)|(load_less_8_days_user_info['2016-04-05']==1)|\
                          (load_less_8_days_user_info['2016-04-04']==1)]

,user_id,sku_id,2016-04-04,2016-04-05,2016-04-06,2016-04-07,2016-04-08,2016-04-09,2016-04-10,load_days,last_buy_time,age,sex,user_lv_cd,user_reg_tm
0,236865,116489,0.0,0.0,0.0,1.0,1.0,1.0,1.0,11.0,2016-01-30,26-35岁,0.0,5,2011-09-26
1,205529,63006,0.0,0.0,0.0,1.0,1.0,1.0,1.0,10.0,2016-01-30,26-35岁,0.0,5,2010-08-27
2,267445,154636,1.0,1.0,0.0,0.0,1.0,0.0,0.0,23.0,2016-02-05,26-35岁,0.0,5,2012-06-28
3,206722,164258,1.0,0.0,1.0,1.0,1.0,1.0,1.0,8.0,2016-01-30,36-45岁,2.0,4,2016-04-02
4,215170,81708,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2016-03-09,36-45岁,2.0,5,2013-09-13
5,249469,154636,0.0,1.0,0.0,1.0,1.0,1.0,1.0,8.0,2016-01-30,26-35岁,1.0,5,2008-03-30
6,237522,78694,0.0,0.0,0.0,0.0,0.0,1.0,1.0,9.0,2016-02-02,36-45岁,0.0,5,2011-10-11
7,207369,154636,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,2016-01-30,16-25岁,2.0,3,2013-05-29
8,249207,63006,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,2016-01-30,26-35岁,1.0,4,2012-02-28
9,214132,9702,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2016-01-30,36-45岁,2.0,4,2013-08-25


In [82]:
# a = buy_date_3[buy_date_3['last_buy_time']=='2016-01-30'] 
# a.sort_values(by='load_days',ascending=False)            366 344  189
# b = test_set_user_load[test_set_user_load.last_buy_time=='2016-01-30']    4
# b = b.sort_values(by='load_days',ascending=False)          


NameError: name 'a' is not defined

In [35]:
# test_user = pd.read_csv('have_cate_8_browse_record.csv')
# with open('./cache/Action_2016-02-01_2016-04-11-load_days','rb') as f:
#     load_days = pickle.load(f)
# with open('./cache/Action_2016-02-01_2016-04-11-user_id-cate-time-type','rb') as f:
#     train_dataset = pickle.load(f)
# load_days['load_days'] = load_days.sum(axis=1)-load_days['user_id']
# test_load_user = pd.merge(test_user,load_days,how='left',on='user_id').dropna(axis=0)
# drop_cols = ['cate_4','cate_5','cate_6','cate_7','cate_9','cate_10','cate_11']
# test_user_info = pd.merge(test_load_user,train_dataset,on='user_id',how='left')
# # #test_user_info['time'] = test_user_info.time.apply(lambda t:t[:10])
# test_user_info = test_user_info.drop(['time','cate','type','sku_id'],axis=1).drop_duplicates()
# test_user_info = test_user_info.drop_duplicates()
# test_user_info_1 = test_user_info[['user_id','cate_8','browse_count','8/count','load_days']]
test_user_info_1.sort_values(by='load_days',ascending=False)

,user_id,cate_8,browse_count,8/count,load_days
149131,242476,388.0,2790.0,0.139068,57.0
8763,204989,407.0,731.0,0.556772,54.0
313216,299166,115.0,999.0,0.115115,48.0
251145,278219,157.0,2907.0,0.054008,47.0
71754,226242,29.0,802.0,0.036160,47.0
141392,239131,170.0,521.0,0.326296,47.0
139121,238956,332.0,678.0,0.489676,44.0
99947,232521,802.0,1174.0,0.683135,43.0
137504,238587,373.0,1071.0,0.348273,42.0
267280,281721,1125.0,2650.0,0.424528,42.0


In [45]:
b=test_user_info_1[test_user_info_1.load_days>7]
b.sort_values(by='8/count',ascending=False)

,user_id,cate_8,browse_count,8/count,load_days
296765,291268,68.0,68.0,1.000000,8.0
317988,301401,111.0,111.0,1.000000,8.0
310493,297595,196.0,196.0,1.000000,16.0
32498,211490,160.0,160.0,1.000000,8.0
219451,268848,207.0,207.0,1.000000,10.0
70771,224918,83.0,83.0,1.000000,9.0
280439,285610,50.0,50.0,1.000000,10.0
120965,235752,176.0,176.0,1.000000,13.0
158923,243636,238.0,238.0,1.000000,16.0
159621,243924,50.0,50.0,1.000000,8.0


In [43]:
test_user_info_1.load_days.describe()

count    1107.000000
mean        7.210479
std         8.051257
min         1.000000
25%         2.000000
50%         4.000000
75%         9.000000
max        57.000000
Name: load_days, dtype: float64

#  以最后5天为测试集，其他为训练集，找规则 5.11

In [14]:
# #将4-11号到4-16所有购买了第八类用户的行为信息全部提取出来
# with open('./cache/Action_2016-02-01_2016-04-16_with_cate-time-type-user_id','rb') as f:
#     all_dataset = pickle.load(f)
# test_dataset = all_dataset[all_dataset.time>'2016-04-11 00:00:00']
# test_user_id = test_dataset[(test_dataset['cate']==8)&(test_dataset['type']==4)]['user_id']
### all_dataset = pd.read_csv('./datafile/JData_Action.csv')[['user_id','time','cate','type']]
# test_set = pd.read_csv('./datafile/test_set.csv')
# test_dataset = pd.read_csv('./datafile/test_dataset.csv')
# with open('./cache/Action_2016-02-01_2016-04-11-load_days','rb') as f:
#     load_days = pickle.load(f)
# with open('./cache/Action_2016-02-01_2016-04-11-user_id-cate-time-type','rb') as f:
#     train_dataset = pickle.load(f)
# test_set_user = test_set['user_id']
# #len(test_set_user)  1385
# test_set_user = pd.DataFrame(test_set_user).drop_duplicates()
# test_set_user 1380    有几个用户买了不止一个第八类商品

In [32]:
#购买了某个用品，购买前些日子有没有查看这个商品
# buy_user_data_1 = pd.merge(test_set,train_dataset,on=['user_id','sku_id'],how='left')
# browse_sku_user_id = buy_user_data_1.dropna(axis=0)
# not_browse_sku_user_id = buy_user_data_1[buy_user_data_1.type.isnull()]

In [17]:
# 找出所有购买了第8类商品的用户在2-01号到4-11号之间的数据
# buy_user_data = pd.merge(test_set,train_dataset,on='user_id',how='left')
# buy_user_data = buy_user_data[['user_id','time','cate','type']].drop_duplicates()
# buy_user_data = buy_user_data.dropna(axis=0)
# groupby_user = buy_user_data[['user_id','cate','type']]
# groupby_user.groupby(by=['user_id','cate'],as_index=True).count()
# type_dummies = pd.get_dummies(groupby_user['cate'],prefix=None)
# groupby_user_1 = groupby_user['user_id']
# groupby_user_1 = pd.DataFrame(groupby_user_1)
# browse_numbers = pd.concat([groupby_user_1,type_dummies],axis=1)
# # browse_numbers2 = browse_numbers[['user_id','4.0','5.0','6.0','7.0','8.0','9.0','10.0','11.0']]
# browse_numbers = browse_numbers.groupby(['user_id'],as_index=False).sum()
# # browse_numbers.drop(['browse_count'],axis=1,inplace=True)
# browse_numbers['browse_count'] = browse_numbers.sum(axis=1)-browse_numbers['user_id']
# browse_numbers.columns = ['user_id','cate_4','cate_5','cate_6','cate_7','cate_8','cate_9','cate_10','cate_11','browse_count']
# browse_numbers['8/count'] = browse_numbers['cate_8'] / browse_numbers['browse_count']
# browse_numbers = browse_numbers.dropna(axis=0)
# have_cate_8_browse_record = browse_numbers[browse_numbers['8/count']!=0]
# have_cate_8_browse_record.to_csv('have_cate_8_browse_record.csv',index=False)
# browse_but_not_cate_8 = browse_numbers[browse_numbers['8/count']==0]
# browse_but_not_cate_8.to_csv('not_cate_8_but_browse.csv',index=False)
# browse_numbers.to_csv('浏览统计表.csv',as_index=False)
# have_cate_8_browse_record['8/count'].describe()

# 目前只考虑这种查看过第8类商品的用户，找出潜在联系

In [27]:
have_cate_8_browse_record = pd.read_csv('have_cate_8_browse_record.csv')
browse_but_not_cate_8 = pd.read_csv('not_cate_8_but_browse.csv')
# outnumber_mean = have_cate_8_browse_record[have_cate_8_browse_record['8/count']>0.636513]
# less_than_mean = have_cate_8_browse_record[have_cate_8_browse_record['8/count']<=0.636513]
# outnumber_mean_2 = outnumber_mean.sort_values(by='8/count',ascending=True)[['user_id','cate_8','browse_count','8/count']]
## outnumber_mean_1 = have_cate_8_browse_record
## outnumber_mean_1.drop(['user_id','browse_count'],axis=1,inplace=True)
## outnumber_mean_1['max'] = outnumber_mean_1.max(axis=1)
## outnumber_mean_1['user_id'] = have_cate_8_browse_record['user_id']
## outnumber_mean_1['browse_count'] = have_cate_8_browse_record['browse_count']
## outnumber_mean_1.columns = ['user_id','cate_4','cate_5','cate_6','cate_7','cate_8','cate_9','cate_10','cate_11','browse_count','8/count','max']

In [29]:
have_cate_8_browse_record

,Unnamed: 0,user_id,cate_4,cate_5,cate_6,cate_7,cate_8,cate_9,cate_10,cate_11,browse_count,8/count
0,0,200034,0.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0,13.0,1.000000
1,1,200333,0.0,0.0,0.0,0.0,3.0,0.0,2.0,0.0,5.0,0.600000
2,3,200794,0.0,0.0,0.0,0.0,21.0,0.0,0.0,0.0,21.0,1.000000
3,8,201252,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,5.0,0.200000
4,9,201285,0.0,0.0,0.0,14.0,5.0,0.0,0.0,0.0,19.0,0.263158
5,11,201391,190.0,5.0,167.0,0.0,20.0,0.0,3.0,0.0,385.0,0.051948
6,12,201529,0.0,12.0,0.0,6.0,5.0,0.0,0.0,0.0,23.0,0.217391
7,13,201591,2.0,0.0,0.0,0.0,48.0,0.0,0.0,0.0,50.0,0.960000
8,15,201672,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,20.0,1.000000
9,16,201740,0.0,0.0,0.0,21.0,134.0,0.0,0.0,0.0,155.0,0.864516


# 查看全集数据，将只查看第8类商品的用户提取出来，再过滤掉最近几天购买了的用户

In [31]:
#将训练集中数据分组，找出符合规则的数据，自己进行测试
# train_dataset_groupby = train_dataset[['user_id','cate']]
# train_dataset_groupby = train_dataset_groupby.groupby(['user_id'],as_index=False).count()
# cate_8_train_dataset = train_dataset[train_dataset.cate==8][['user_id','cate','type']]  
# cate_8_train_dataset = cate_8_train_dataset.groupby(['user_id'],as_index=False).count()
# cate_8_train_dataset = cate_8_train_dataset.drop(['cate'],axis=1)
# train_dataset_groupby = pd.merge(train_dataset_groupby,cate_8_train_dataset,on='user_id',how='left')
# train_dataset_groupby.columns = ['user_id','browse_counts','browse_cate_8']
# train_dataset_groupby['cate_8_ratio'] = train_dataset_groupby['browse_cate_8'] / train_dataset_groupby['browse_counts']
# ratio_accute_1 = train_dataset_groupby[train_dataset_groupby['cate_8_ratio']==1]
# ratio_accute_1


,user_id,sku_id,time,cate,type
0,200006.0,84389,2016-03-15 18:55:59,8,6
1,200006.0,84389,2016-03-15 18:55:59,8,1
2,200006.0,84389,2016-03-15 18:55:59,8,6
3,200006.0,84389,2016-03-15 18:55:59,8,1
4,200006.0,84389,2016-03-15 18:56:06,8,1
5,200006.0,84389,2016-03-15 18:56:06,8,6
6,200006.0,84389,2016-03-15 18:56:06,8,1
7,200006.0,84389,2016-03-15 18:56:06,8,6
8,200006.0,84389,2016-03-15 18:56:23,8,1
9,200006.0,84389,2016-03-15 18:56:23,8,1


#  统计ratio_accute_1的用户登录情况

In [70]:
# ratio_accute_1_user = pd.DataFrame(ratio_accute_1['user_id'])
# ratio_accute_1_user = pd.merge(ratio_accute_1_user,train_dataset,on='user_id',how='left')
# ratio_accute_1_user['time'] =  ratio_accute_1_user.time.apply(lambda t:t[:10])
# ratio_accute_1_user_and_time = ratio_accute_1_user[['user_id','time']].drop_duplicates()
# type_dummies = pd.get_dummies(ratio_accute_1_user_and_time['time'],prefix=None)
# ratio_accute_1_user_and_time = pd.concat([ratio_accute_1_user_and_time,type_dummies],axis=1)
# ratio_accute_1_user_and_time = ratio_accute_1_user_and_time.drop(['time'],axis=1)
# ratio_accute_1_user_load = ratio_accute_1_user_load.drop(['load_days'],axis=1)
# ratio_accute_1_user_load = ratio_accute_1_user_and_time.groupby(['user_id'],as_index=False).sum()
# ratio_accute_1_user_load['load_days'] = ratio_accute_1_user_load.sum(axis=1) - ratio_accute_1_user_load['user_id']
temp_user = pd.merge(have_cate_8_browse_record,ratio_accute_1_user_load,on='user_id',how='left')
temp_1 = temp_user.dropna(axis=0)
temp_1 = temp_1.drop(['Unnamed: 0','cate_4','cate_5','cate_6','cate_7','cate_10','cate_9','cate_11','cate_8','8/count'],axis=1)
temp_1.sort_values('load_days',ascending=False)

,user_id,browse_count,2016-01-31,2016-02-01,2016-02-02,2016-02-03,2016-02-04,2016-02-05,2016-02-06,2016-02-07,...,2016-04-02,2016-04-03,2016-04-04,2016-04-05,2016-04-06,2016-04-07,2016-04-08,2016-04-09,2016-04-10,load_days
1042,297595,196.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,16.0
485,243636,238.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,16.0
386,235752,176.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,13.0
924,285610,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,10.0
741,268848,207.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,10.0
255,224918,83.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,9.0
488,243924,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,8.0
131,211490,160.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,8.0
984,291268,68.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0
1073,301401,111.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,8.0


In [65]:
temp_user = pd.merge(have_cate_8_browse_record,ratio_accute_1_user_load,on='user_id',how='left')
temp_1 = temp_user.dropna(axis=0)
temp_1 = temp_1.drop('Unnamed: 0',axis=1)
temp_1.sort_values('load_days',ascending=False)



,user_id,cate_4,cate_5,cate_6,cate_7,cate_8,cate_9,cate_10,cate_11,browse_count,...,2016-04-02,2016-04-03,2016-04-04,2016-04-05,2016-04-06,2016-04-07,2016-04-08,2016-04-09,2016-04-10,load_days
0,200034,0.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,200794,0.0,0.0,0.0,0.0,21.0,0.0,0.0,0.0,21.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,2.0
2,201672,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,20.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,202215,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,202284,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,18.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0
5,202508,0.0,0.0,0.0,0.0,39.0,0.0,0.0,0.0,39.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0
6,203588,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,9.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0
7,204281,0.0,0.0,0.0,0.0,13.0,0.0,0.0,0.0,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0
8,204985,0.0,0.0,0.0,0.0,21.0,0.0,0.0,0.0,21.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
9,205252,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


# 将测试集中的用户数据完整呈现出来

In [2]:
# have_cate_8_user = have_cate_8_browse_record['user_id']
# have_cate_8_user = pd.DataFrame(have_cate_8_user)
# have_cate_8 = pd.merge(have_cate_8_user,train_dataset,how='left',on='user_id')
# have_cate_8.groupby('user_id',as_index=False).time.max()

# 给02-01号到04-11号所有购买过第8类商品的用户提纯

In [20]:
# type_4_train_dataset = cate_8_train_dataset[cate_8_train_dataset['type']==4]
# type_4_train_dataset

,user_id,type
78,200079.0,4
552,200561.0,4
735,200747.0,4
953,200969.0,4
1223,201244.0,4
1732,201766.0,4
1755,201789.0,4
1764,201798.0,4
1828,201862.0,4
1897,201931.0,4


In [96]:
import matplotlib.pyplot as plt


In [99]:
# have_cate_8_browse_record.plot('browse_count','cate_8')
# have_cate_8_browse_record.plot('cate_8','browse_count')
# plt.show()

In [56]:
# only_browse_cate_8 = groupby_user[(groupby_user.cate==4)|(groupby_user.cate==5)|(groupby_user.cate==6)\
#              |(groupby_user.cate!=7)|(groupby_user.cate!=9)|(groupby_user.cate==10)&(groupby_user.cate==8)]['user_id']
# only_browse_cate_8 = pd.DataFrame(only_browse_cate_8).drop_duplicates()
# groupby_user.drop(groupby_user['user_id']==only_browse_cate_8['user_id'],axis=0)

In [12]:
test_dataset

,user_id,time,cate,type
0,236865.0,2016-04-11 00:00:20,5,6
1,236865.0,2016-04-11 00:00:20,5,1
2,236865.0,2016-04-11 00:00:20,5,1
3,236865.0,2016-04-11 00:00:20,5,6
4,236865.0,2016-04-11 00:01:00,8,4
5,236865.0,2016-04-11 00:01:00,5,4
6,236865.0,2016-04-11 00:06:19,5,1
7,236865.0,2016-04-11 00:06:19,5,6
8,236865.0,2016-04-11 00:06:19,5,6
9,236865.0,2016-04-11 00:06:19,5,1


In [17]:
# all_dataset.sort_values(by='time',ascending=False)

,cate,time,type,user_id
21299970,6,2016-03-12 19:19:36,1,277214.0
21299963,8,2016-03-12 19:19:35,1,243568.0
21299956,5,2016-03-12 19:19:35,1,209107.0
21299957,8,2016-03-12 19:19:35,1,243568.0
21299958,5,2016-03-12 19:19:35,1,209107.0
21299959,8,2016-03-12 19:19:35,6,243568.0
21299960,5,2016-03-12 19:19:35,6,209107.0
21299961,5,2016-03-12 19:19:35,6,209107.0
21299962,8,2016-03-12 19:19:35,1,293060.0
21299964,8,2016-03-12 19:19:35,1,243568.0
